<a href="https://colab.research.google.com/github/itsGab/projeto-bootcamp-modulo-2-data-science/blob/main/nb-limpeza-dados/limpeza_casos_fa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpeza: Casos de Febre Amarela

Importações

In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date

Leitura dos dados

In [25]:
# 2001-2006
url_c1 = 'https://github.com/itsGab/projeto-bootcamp-modulo-2-data-science/raw/main/dados-brutos/casos%20confirmados%20por%20uf%20de%20notificacao%20segundo%20ano%20de%20atendimento%20de%202001%20a%202006.csv'

# 2007-2016
url_c2 = 'https://github.com/itsGab/projeto-bootcamp-modulo-2-data-science/raw/main/dados-brutos/casos%20confirmados%20por%20uf%20de%20notificacao%20segundo%20ano%20de%20atendimento%20de%202007%20a%202016.csv'


In [26]:
# lendo o arquivo csv
df1 = pd.read_csv(url_c1, sep=';', encoding='latin-1', engine='python', decimal=',',
            thousands='.', skiprows=3, skipfooter=14)

In [28]:
# limpando os dados
df1 = df1.drop('Total', axis=1)
df1.columns = df1.columns.str.lower()
lista = df1.columns.to_list()
lista[0] = 'ano'
df1.columns = lista 
df1 = df1.replace('-', 0).astype(int)
df1['ano'] = df1['ano'].map({ano: date(ano,1,1) for ano in df1['ano']})

In [29]:
# lendo o arquivo csv
df2 = pd.read_csv(url_c2, sep=';', encoding='latin-1', engine='python', decimal=',',
            thousands='.', skiprows=3, skipfooter=13)

In [30]:
# limpando os dados
df2.columns = df2.columns.str.lower()
lista = df2.columns.tolist()
lista[0]= ('ano')
df2.columns = lista
df2 = df2.drop('total', axis=1)
df2 = df2.replace('-', 0).astype(int)
df2['ano'] = df2['ano'].map({ano: date(ano,1,1) for ano in df2['ano']})

In [31]:
# lista de todos os estados
lista_uf = ['ano', 'ro', 'ac', 'am', 'rr', 'pa', 'ap', 'to', 'ma', 'pi', 'ce', 
            'rn', 'pb', 'pe', 'al', 'se', 'ba', 'mg', 'es', 'rj', 'sp', 'pr', 
            'sc', 'rs', 'ms', 'mt', 'go', 'df']

In [32]:
# adicionando as linhas dos estados que nao tiveram casos df1
for uf in lista_uf:
    if uf not in df1.columns:
        df1[uf] = [0 for n in range(len(df1))]

df1 = df1.loc[:, lista_uf]

In [33]:
# adicionando as linhas dos estados que nao tiveram casos df2
for uf in lista_uf:
    if uf not in df2.columns:
        df2[uf] = [0 for n in range(len(df2))]

df2 = df2.loc[:, lista_uf]

In [34]:
# juntando os dois df
df3 = df2.set_index('ano').T
df4 = df1.set_index('ano').T
df5 = pd.merge(df3, df4, left_index=True, right_index=True)
df5['2006-01-01'] = df5['2006-01-01_x'] + df5['2006-01-01_y']
df5 = df5.drop(['2006-01-01_x', '2006-01-01_y'], axis=1).T.reset_index()
df5['ano'] = df5['ano'].astype('datetime64')
df5 = df5.set_index('ano').sort_index().reset_index()


In [53]:
# df melt
df5_melt = df5.melt(id_vars='ano', var_name='uf', value_name='casos')


In [36]:
# salvando os dados tratados
df5.to_csv('casos_fa_pivot.csv')
df5_melt.to_csv('casos_fa_melt.csv')
